<a href="https://colab.research.google.com/github/Jl16ExA/Surya-OCR-Hardware-Benchmarking/blob/main/Surya_OCR_Benchmarking_Across_Different_GPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Surya-OCR-Benchmarking Across Different GPUs

This notebook is dedicated to benchmarking the performance of the Surya OCR system across various hardware setups, focusing on GPU utilization and scalability.

This is still an early version I made in a Friday evening only testing a single image being processed in different batch sizes.

If you have any questions or wish to collaborate, feel free to reach out via my social media platforms. I am always eager to discuss and help.

- Developer: Juan David López
- GitHub: [Jl16ExA](https://github.com/Jl16ExA/find_your_class_javeriana)
- LinkedIn:  [Juan David López Becerra](https://www.linkedin.com/in/juan-david-lopez-becerra-5048271bb/)
- Twitter:[ @JLopez_160](https://twitter.com/JLopez_160)

### Dependencies

In [ ]:
!pip install py-cpuinfo
!pip install GPUtil
!pip install surya-ocr
!pip install pdf2image Pillow reportlab
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

### Load Model

In [ ]:
from PIL import Image
from surya.ocr import run_ocr
from surya.model.detection.segformer import load_model as load_det_model, load_processor as load_det_processor
from surya.model.recognition.model import load_model as load_rec_model
from surya.model.recognition.processor import load_processor as load_rec_processor

langs = ["es"] # Replace with your languages
det_processor, det_model = load_det_processor(), load_det_model()
rec_model, rec_processor = load_rec_model(), load_rec_processor()

### Run Benchmark

In [ ]:
import os
import time
import pandas as pd
from PIL import Image
import platform
import cpuinfo
import GPUtil
import psutil

# Define standard deep learning batch sizes to test
batch_sizes = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

# Create a DataFrame to store results
results = pd.DataFrame()

# Load and preprocess the image
image_path = "path/to/png"
image = Image.open(image_path)
image = image.resize((2048, int(image.height * 2048 / image.width)))

def get_system_resources():
    gpus = GPUtil.getGPUs()
    if gpus:
        gpu = gpus[0]
        gpu_name = gpu.name
        total_gpu_memory = gpu.memoryTotal
        free_gpu_memory = gpu.memoryFree
        used_gpu_memory = gpu.memoryUsed
    else:
        gpu_name = 'No GPU detected'
        total_gpu_memory = free_gpu_memory = used_gpu_memory = 0

    virtual_memory = psutil.virtual_memory()
    total_ram = virtual_memory.total / (1024 ** 3)  # Convert from Bytes to GB
    available_ram = virtual_memory.available / (1024 ** 3)
    used_ram = virtual_memory.used / (1024 ** 3)

    return {
        'CPU': cpuinfo.get_cpu_info()['brand_raw'],
        'OS': platform.system() + " " + platform.release(),
        'GPU': gpu_name,
        'Total GPU Memory (GB)': total_gpu_memory / 1024,  # Convert MB to GB
        'Free GPU Memory (GB)': free_gpu_memory / 1024,
        'Used GPU Memory (GB)': used_gpu_memory / 1024,
        'Total RAM (GB)': total_ram,
        'Available RAM (GB)': available_ram,
        'Used RAM (GB)': used_ram
    }

memory_error_occurred = False

for batch_size in batch_sizes:
    os.environ['RECOGNITION_BATCH_SIZE'] = str(batch_size)
    images = [image.copy() for _ in range(batch_size)]
    print(f"Running batch size: {batch_size}")
    if not memory_error_occurred:
        try:
            start_time = time.time()
            predictions_ocr = run_ocr(images, [langs]*batch_size, det_model, det_processor, rec_model, rec_processor)  # Define these variables and function
            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Batch Size {batch_size}: Completed in {elapsed_time:.2f} seconds.")
        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                elapsed_time = None
                memory_error_occurred = True
                print(f"Batch Size {batch_size} failed: CUDA out of memory.")
            else:
                raise  # Re-raise the exception if it's not a memory error
    else:
        elapsed_time = None
        print(f"Skipping Batch Size {batch_size} due to previous CUDA out of memory error.")

    system_resources = get_system_resources()
    system_resources.update({
        'Batch Size': batch_size,
        'Elapsed Time': elapsed_time,
        'Error': 'CUDA out of memory' if elapsed_time is None else None
    })
    new_row = pd.DataFrame([system_resources])
    results = pd.concat([results, new_row], ignore_index=True)
    results.to_csv('ocr_benchmark_results.csv', index=False)

print("Benchmarking completed. Results saved to 'ocr_benchmark_results.csv'.")
